In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.chrome.options import Options

# Set Paths
CHROMEDRIVER_PATH = "C:/BraveDriver/chromedriver.exe"  
BRAVE_PATH = "C:/Program Files/BraveSoftware/Brave-Browser/Application/brave.exe"  

# Setup Brave WebDriver
options = Options()
options.binary_location = BRAVE_PATH  
options.add_argument("--headless")  
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")

# Start WebDriver
service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=options)

# Open YouTube Trending Page
url = "https://www.youtube.com/results?search_query=finance&sp=CAM%253D"
driver.get(url)
time.sleep(5)  # Allow time to load

# Scroll multiple times to load more videos
for _ in range(5):
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(3)

# Parse with BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")
videos = soup.find_all("ytd-video-renderer", class_="style-scope ytd-item-section-renderer")

# Extract Video Data
data = []
for video in videos:
    # Title
    title_tag = video.find("a", id="video-title")
    title = title_tag.text.strip() if title_tag else "No Title"

    # Views
    views_tag = video.find_all("span", class_="inline-metadata-item style-scope ytd-video-meta-block")
    views = views_tag[0].text.strip().replace("views", "").replace(",", "") if views_tag else "0"

    # Channel Name
    channel_tag = video.find("a", class_="yt-simple-endpoint style-scope yt-formatted-string")
    channel = channel_tag.text.strip() if channel_tag else "Not Found"

    # Published Date
    publish_tag = video.find("div", id="metadata-line")
    published_date = publish_tag.text.strip() if publish_tag else "Unknown"

    # Video Duration
    duration_tag = video.find("span", class_="style-scope ytd-thumbnail-overlay-time-status-renderer")
    duration = duration_tag.text.strip() if duration_tag else "Unknown"

    # Likes (JavaScript Execution Required)
    try:
        like_button = video.find("yt-formatted-string", class_="style-scope ytd-toggle-button-renderer")
        likes = like_button["aria-label"].split(" ")[0] if like_button else "Unknown"
    except:
        likes = "Unknown"

    # Comments (JavaScript Execution Required)
    try:
        comment_section = video.find("yt-formatted-string", class_="count-text style-scope ytd-comments-header-renderer")
        comments = comment_section.text.strip().split(" ")[0] if comment_section else "Unknown"
    except:
        comments = "Unknown"

    data.append([title, views, channel, published_date, duration, likes, comments])

# Close Browser
driver.quit()

# Save Data to CSV
df = pd.DataFrame(data, columns=["Title", "Views", "Channel", "Published Date", "Duration", "Likes", "Comments"])
df.to_csv("youtube_finance_trending.csv", index=False)

print("✅ Scraping Complete! Data saved as 'youtube_finance_trending.csv'")


✅ Scraping Complete! Data saved as 'youtube_finance_trending.csv'


In [3]:
df.head(10)

,Title,Views,Channel,Published Date,Duration,Likes,Comments
0,Why You Should Invest Early,64M,Finance With Sharan,•\n64M views\n1 year ago,SHORTS,Unknown,Unknown
1,📈 The Power of Compound Interest #shorts,64M,Sean Pan,•\n64M views\n2 years ago,SHORTS,Unknown,Unknown
2,Petrol Pump Scam⛽️ #finance #petrol #education,59M,Tejas Joshi Finance,•\n59M views\n2 years ago,SHORTS,Unknown,Unknown
3,Don’t Pay Toll If Toll Plaza? #finance #money ...,53M,Basic Gyaan,•\n53M views\n5 months ago,SHORTS,Unknown,Unknown
4,How The Economic Machine Works by Ray Dalio,42M,Principles by Ray Dalio,•\n42M views\n11 years ago,31:00,Unknown,Unknown
5,How much money is in your bank account? 🤔💰 #...,41M,Chris Stocks,•\n41M views\n2 years ago,SHORTS,Unknown,Unknown
6,Know Your Finance Pt. 4 #financewithsharan #sh...,40M,Finance With Sharan,•\n40M views\n1 year ago,SHORTS,Unknown,Unknown
7,How to avoid student loans 101 👏🏾 #shorts #fin...,39M,JC Rodriguez,•\n39M views\n2 years ago,SHORTS,Unknown,Unknown
8,How much money is in your bank account? 🤔💰 #sh...,37M,Chris Stocks,•\n37M views\n2 years ago,SHORTS,Unknown,Unknown
9,Phyno - Financial Woman (Official Music Video)...,38M,PhynoFino,•\n38M views\n8 years ago,4:30,Unknown,Unknown
